In [1]:
import os
import glob
import numpy as np
import pandas as pd

import nibabel as nib
from nibabel.viewers import OrthoSlicer3D
from collections import Counter

In [2]:
test_path = glob.glob("data/Test/*")
train_path = glob.glob("data/Train/*/*")
np.random.shuffle(train_path)
np.random.shuffle(test_path)
print(train_path)

['data/Train\\NC\\2.nii', 'data/Train\\MCI\\13.nii', 'data/Train\\MCI\\7.nii', 'data/Train\\NC\\17.nii', 'data/Train\\NC\\23.nii', 'data/Train\\MCI\\21.nii', 'data/Train\\MCI\\18.nii', 'data/Train\\MCI\\10.nii', 'data/Train\\NC\\22.nii', 'data/Train\\MCI\\19.nii', 'data/Train\\MCI\\3.nii', 'data/Train\\NC\\21.nii', 'data/Train\\NC\\16.nii', 'data/Train\\NC\\19.nii', 'data/Train\\MCI\\14.nii', 'data/Train\\MCI\\23.nii', 'data/Train\\MCI\\6.nii', 'data/Train\\NC\\13.nii', 'data/Train\\MCI\\4.nii', 'data/Train\\MCI\\20.nii', 'data/Train\\NC\\11.nii', 'data/Train\\MCI\\5.nii', 'data/Train\\NC\\6.nii', 'data/Train\\MCI\\16.nii', 'data/Train\\NC\\3.nii', 'data/Train\\MCI\\17.nii', 'data/Train\\MCI\\9.nii', 'data/Train\\NC\\10.nii', 'data/Train\\MCI\\24.nii', 'data/Train\\NC\\4.nii', 'data/Train\\NC\\15.nii', 'data/Train\\NC\\12.nii', 'data/Train\\NC\\14.nii', 'data/Train\\NC\\25.nii', 'data/Train\\MCI\\8.nii', 'data/Train\\MCI\\12.nii', 'data/Train\\MCI\\25.nii', 'data/Train\\MCI\\2.nii', 'd

In [3]:
def get_features(path):
    img = nib.load(path)
    img = img.dataobj[:,:,:,0]
    random_img = img[:,:,np.random.choice(range(img.shape[2]),10)] # ->随机取十个样本

    feat = [
        (random_img != 0 ).sum(),   # ->非零像素的数量
        (random_img == 0 ).sum(),   # ->零像素的数量
        random_img.mean(),      # ->平均值
        random_img.std(),       # ->标准差
        len(np.where(random_img.mean(0))[0]),   #->列方向上平均值不为零的数量
        len(np.where(random_img.mean(1))[0]),   #->行方向上平均值不为零的数量
        random_img.mean(0).max(),       #->列方向上最大平均值
        random_img.mean(1).max()        #->行方向上最大平均值
    ]

    if "NC" in path:
        return feat + ["NC"]
    else:
        return feat + ["MCI"]

In [5]:
train_feat = []
for _ in range(30):
    for path in train_path:
        train_feat.append(get_features(path))

test_feat = []
for _ in range(30):
    for path in test_path:
        test_feat.append(get_features(path))

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10)
model.fit(
    np.array(train_feat)[:,:-1].astype(np.float32),
    np.array(train_feat)[:,-1]
    )


C:\Users\34778\anaconda3\envs\for_ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10)

In [10]:
test_pred = model.predict(np.array(test_feat)[:,:-1].astype(np.float32))
test_pred = test_pred.reshape(30,-1).T

test_pred_label = [Counter(x).most_common(1)[0][0] for x in test_pred]

submit = pd.DataFrame(
    {
        'uuid':[int(x.split('\\')[-1][:-4]) for x in test_path],
        'label':test_pred_label

    }
)

submit = submit.sort_values(by='uuid')
submit.to_csv('submit.csv',index=None)